# Data Mining OCR PDFs - Using *pdftabextract* to liberate tabular data from scanned documents

This is an example on how to use *pdftabextract* for data mining in scanned and OCR-processed documents with rather complex tables and/or few "optical features" like column or row borders, which often make it impossible to use like tools like [Tabula](http://tabula.technology/).

In this example, only a table from a single page will extracted for demonstration purposes. For a full example that covers several pages, see the `catalog_30s.py` script. There are also some more demonstrations in the `examples` directory.

The page that we will process looks like this:

![PDF Excerpt p. 3](data/ALA1934_RR-excerpt.pdf-3_1.png)

The page has been scanned and processed with Optical Character Recognition (OCR) software like *ABBYY FineReader* or *tesseract* and produced a "sandwich" PDF with the scanned document image and the recognized text boxes. Although some software, like FineReader allows to extract tables, this often fails and some more effort in order to liberate the data. We can use *pdftabextract* together with some other other tools for this.

We will use a combination of the following tools in order to reach our goal:

* the `pdftohtml` command from *poppler-utils* to extract the texts and scanned images from the PDF
* *pdf2xml-viewer* to inspect the text boxes and the generated table grid (more on that later)
* `pdftabextract` to write a script estimates the positions of columns and rows, generates a table grid and fits the text boxes into this grid in order to extract the tabular data

The following steps will be performed and described in detail:

1. Extract the scanned page images and generate an XML with the OCR texts of the PDF with `pdftohtml`
2. View the text boxes and scanned pages with *pdf2xml-viewer*
3. Load the XML describing the pages and text boxes
4. Detect clusters of vertical lines using the image processing module of `pdftabextract`
5. Find page rotation or skew and fix it
6. Get column and line positions
7. Create a grid of columns and lines
8. Match the text boxes into the grid and hence extract the tabular data in order to export it as Excel and CSV file

## 1. Extract the scanned page images and generate an XML with the OCR texts of the PDF with `pdftohtml`

The tool `pdftohtml` is part of the software package *[poppler-utils](https://poppler.freedesktop.org/)*. It's available on most Linux distributions and also for OSX via Homebrew or MacPorts. Using the argument `-xml`, it will generate an XML file in *pdf2xml* format with from a sandwich PDF:

In [9]:
!cd data/ && pdftohtml -c -hidden -xml ALA1934_RR-excerpt.pdf ALA1934_RR-excerpt.pdf.xml

Page-1
Page-2
Page-3
Page-4


In [10]:
!ls -1 data/

ALA1934_RR-excerpt.pdf
ALA1934_RR-excerpt.pdf-1_1.png
ALA1934_RR-excerpt.pdf-2_1.png
ALA1934_RR-excerpt.pdf-3_1.png
ALA1934_RR-excerpt.pdf-4_1.png
ALA1934_RR-excerpt.pdf.xml


In [11]:
!head -n 30 data/ALA1934_RR-excerpt.pdf.xml

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE pdf2xml SYSTEM "pdf2xml.dtd">

<pdf2xml producer="poppler" version="0.41.0">
<page number="1" position="absolute" top="0" left="0" height="1261" width="892">
	<fontspec id="0" size="23" family="Times" color="#000000"/>
	<fontspec id="1" size="41" family="Times" color="#000000"/>
	<fontspec id="2" size="20" family="Times" color="#000000"/>
	<fontspec id="3" size="22" family="Times" color="#000000"/>
	<fontspec id="4" size="7" family="Times" color="#000000"/>
	<fontspec id="5" size="7" family="Times" color="#000000"/>
	<fontspec id="6" size="13" family="Times" color="#000000"/>
	<fontspec id="7" size="5" family="Times" color="#000000"/>
	<fontspec id="8" size="13" family="Times" color="#000000"/>
	<fontspec id="9" size="4" family="Times" color="#000000"/>
	<fontspec id="10" size="4" family="Times" color="#000000"/>
	<fontspec id="11" size="5" family="Times" color="#000000"/>
	<fontspec id="12" size="7" family="Times" color="#000000"/>
	<fo

We generated an XML which consists of several `<page>` elements, containing an `<image>` (the "background" image, i.e. the scanned page) and several text boxes (`<text>`) with coordinates and the respective text box value. The images themselves have also been extracted from the PDF.

## 2. View the text boxes and scanned pages with *pdf2xml-viewer*

You can download the [pdf2xml-viewer from its github page](https://github.com/WZBSocialScienceCenter/pdf2xml-viewer). It basically consists of an HTML page that allows you to inspect an XML file in pdf2xml format in your browser. A copy of it resides also in the directory of this example.

Change to the directory where pdf2xml-viewer resides (where its *index.html* or *pdf2xml-viewer.html* file is). You should also copy the extracted XML file and images to this location. Now let's start up a minimal local webserver. This can be done very easily with Python:

With Python 2.x: `python -m SimpleHTTPServer 8080`

Or with Python 3: `python3 -m http.server 8080 --bind 127.0.0.1`

Now you open your browser and go to the adress http://127.0.0.1:8080. The viewer shows up and you can now enter the
file name of your file to load (it must be relative to the directory in which pdf2xml-viewer resides).

In [ ]:
!python3 -m http.server 8080 --bind 127.0.0.1

Serving HTTP on 127.0.0.1 port 8080 ...


When you execute the above line you can start up pdf2xml-viewer by visiting http://127.0.0.1:8080/pdf2xml-viewer.html. **However, please note that this will prevent further code execution in this Notebook as long as the minimal webserver is running.** So it's generally a better idea to execute this in a separate Terminal window.

In pdf2xml-viewer you will now be able to browse through the pages. In the background, you can see the scanned image page and on top of that are the text boxes that were detected during OCR:

![pdf2xml-viewer p.3 displayed](nb_images/pdf2xml-viewer-page.png)

## 3. Load the XML describing the pages and text boxes

We can now start to use `pdftabextract` in Python code in order to load the XML file. Let's define some constants first that we will need throughout the script.

In [3]:
DATAPATH = 'data/'
OUTPUTPATH = 'generated_output/'
INPUT_XML = 'ALA1934_RR-excerpt.pdf.xml'

Now we can load the the XML, parse it and have a look at the third page (the page from which we later want to extract the data).

In [4]:
import os
from pdftabextract.common import read_xml, parse_pages

# Load the XML that was generated with pdftohtml
xmltree, xmlroot = read_xml(os.path.join(DATAPATH, INPUT_XML))

# parse it and generate a dict of pages
pages = parse_pages(xmlroot)

In [7]:
pages[3].keys()

dict_keys(['xmlnode', 'number', 'height', 'image', 'width', 'texts'])

Each page consists of an *xmlnode* which points to the original XML page element, a page number, the page dimensions, an image (the scanned page) and the text boxes:

In [11]:
from pprint import pprint

p_num = 3
p = pages[p_num]

print('number', p['number'])
print('width', p['width'])
print('height', p['height'])
print('image', p['image'])
print('the first three text boxes:')
pprint(p['texts'][:3])

number 3
width 892
height 1261
image ALA1934_RR-excerpt.pdf-3_1.png
the first three text boxes:
[{'bottom': 107.0,
  'bottomleft': array([ 359.,  107.]),
  'bottomright': array([ 404.,  107.]),
  'height': 14,
  'left': 359.0,
  'right': 404.0,
  'top': 93.0,
  'topleft': array([ 359.,   93.]),
  'topright': array([ 404.,   93.]),
  'value': 'Baden',
  'width': 45,
  'xmlnode': <Element 'text' at 0x7fb36590e9a8>},
 {'bottom': 104.0,
  'bottomleft': array([ 737.,  104.]),
  'bottomright': array([ 745.,  104.]),
  'height': 13,
  'left': 737.0,
  'right': 745.0,
  'top': 91.0,
  'topleft': array([ 737.,   91.]),
  'topright': array([ 745.,   91.]),
  'value': '3',
  'width': 8,
  'xmlnode': <Element 'text' at 0x7fb36590e9f8>},
 {'bottom': 144.0,
  'bottomleft': array([  58.,  144.]),
  'bottomright': array([  76.,  144.]),
  'height': 8,
  'left': 58.0,
  'right': 76.0,
  'top': 136.0,
  'topleft': array([  58.,  136.]),
  'topright': array([  76.,  136.]),
  'value': 'Ein\xad',
  'width

The text boxes in `p['texts']` contain the coordinates and dimensions of each text box as well as the content (`value`) and a reference to the original XML node.